In [11]:
import numpy as np
import six
import sys
import os
import traceback
import re
import pickle


from chainer import cuda
from context2vec.common.context_models import Toks
from context2vec.common.model_reader import ModelReader
import sklearn
import pandas as pd
import logging
from scipy.stats import spearmanr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
import math


[nltk_data] Downloading package stopwords to /home/ql261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def produce_top_n_simwords(w_filter,context_embed,n_result,index2word):
        # compute top n_result similarity weights
        similarity_scores=[]
        print('producing top {0} simwords'.format(n_result))
        similarity = (w_filter.dot(context_embed)+1.0)/2
        top_words_i=[]
        count = 0
        for i in (-similarity).argsort():
                    if xp.isnan(similarity[i]):
                        continue
                    print('{0}: {1}'.format(str(index2word[i]), str(similarity[i])))
                    count += 1
                    top_words_i.append(i)
                    similarity_scores.append(similarity[i])
                    if count == n_result:
                        break

        top_vec=w_filter[top_words_i,:]
        
        return top_vec,similarity_scores
    
def top_mutual_sim(top_vec,similarity_scores):
#     a=np.array([1,2,3])
# b=a.reshape(len(a),1)
# c=(a+b)/2.0
# d=np.array([[1,1,1],[2,2,2],[3,3,3]])
# e=d.dot(d.T)
# print ('e',e)
# print ('c',c)
# e*c
    max_score=similarity_scores[0]
    similarity_scores=np.array(similarity_scores)
#     similarity_scores=similarity_scores/sum(similarity_scores)
    sim_weights=(similarity_scores+similarity_scores.reshape(len(similarity_scores),1))/2.0
    sim_weights=(sim_weights/float(sum(sum(sim_weights))))*max_score
    inf_score=sum(sum(top_vec.dot(top_vec.T)*sim_weights))
    return inf_score

In [35]:
# def space_punc(sent):
#     new_sent=sent
#     add=0
#     for m in re.finditer(r'[\d\-\/\"\';,.!?\)\(]+',sent):
#         if m.end(0)+add>=len(new_sent):
#             return new_sent[:m.start(0)+add]+' '+new_sent[m.start(0)+add:m.end(0)+add]
#         else:
#             new_sent=new_sent[:m.start(0)+add]+' '+new_sent[m.start(0)+add:m.end(0)+add]+' '+new_sent[m.end(0)+add:]
#             add+=2
#     return new_sent
        
# def mult_sim(w, target_v, context_v):
#     target_similarity = w.dot(target_v)
#     target_similarity[target_similarity<0] = 0.0
#     context_similarity = w.dot(context_v)
#     context_similarity[context_similarity<0] = 0.0
#     return (target_similarity * context_similarity)


# def find_unk_pos(words,test_w):
#     try:
#         pos=words.index(test_w.upper())
#     except ValueError:
#         try:
#             pos=words.index(test_w.upper()+'S')
#         except ValueError as e:
#             try:
#                 pos=words.index(test_w.upper()+'ES')
#             except ValueError as e:
#                 try:
#                     pos=words.index(test_w.upper()[:-1]+'IES')
#                 except ValueError as e:
#                     try:
#                         pos=words.index(test_w.upper()+test_w.upper()[-1]+'ES')
#                     except ValueError as e:
#                         print (e)
#     return pos


    
def skipgram_context(model,words,pos,weight=None,w2entropy=None):
    context_wvs=[]
    weights=[]
    for i,word in enumerate(words):
        if i != pos:
            
            try:
                if weight=='lda' :
                
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(1.0/float(w2entropy[word]))
                        context_wvs.append(model[word])
                else:

                    context_wvs.append(model[word])
                    weights.append(1.0)
            except KeyError as e:
                print ('==warning==: key error in context {0}'.format(e))
    context_embed=sum(np.array(context_wvs)*np.array(weights).reshape(len(weights),1))/sum(weights)
    return len(context_wvs),context_embed


def context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy=None):
    test_s=test_s.replace(test_w, ' '+test_w+' ')
    
    #test_s=space_punc(test_s)
    print(test_s)
    words=test_s.split()
    pos=words.index(test_w)
    
    
    score=1.0
    #decide on model
    if model_type=='context2vec':
        
        context_embed= model.context2vec(words, pos)
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
    
    elif model_type=='skipgram':
        score,context_embed=skipgram_context(model,words,pos,weight,w2entropy)
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
        
        
    else:
        print ('model type {0} not recognized'.format(model_type))
        sys.exit(1)
        
        
    #decide on weight 
    if weight=='top_mutual_sim':
        top_vec,sim_scores=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
#         score=sum(sum(top_vec.dot(top_vec.T)))/(n_result**2)
        score=top_mutual_sim(top_vec,sim_scores)
        if model_type=='skipgram':
            score=score*score
    elif weight=='learned':
        print ('learned not implemented')
    elif weight=='gaussian':
        print ('gaussian not implemented')
    elif weight ==False:
        score=score
    else:
        print ('weight mode {0} not recognized'.format(weight))
    return score,context_embed

def additive_model(f_w,test_ss,test_w, model_type,model,n_result,w_filter,index2word,weight=False,w2entropy=None):
    
    context_out=[]
    context_weights=[]
    for test_s in test_ss.split('@@'):
        test_s=test_s.strip()
        score,context_embed=context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy)
        print ('weight is {0}'.format(score))
        context_out.append(context_embed)
        context_weights.append(score)
    context_out=np.array(context_out)
    norm_weights=np.array(context_weights).reshape(len(context_weights),1)/float(sum(context_weights))
    #f_w.write(','.join(list(norm_weights.reshape(1, len(context_weights))[0])))
    print ('normalized weight: \n  {0}'.format(norm_weights))
    context_avg=sum(norm_weights*context_out)
    print('producing top {0} words for new embedding'.format(n_result))
    top_vec,scores=produce_top_n_simwords(w_filter,context_avg,n_result,index2word)
#     f_w.write(','.join([str(w) for w in list(norm_weights.reshape(1, len(context_weights))[0]))]))
#     np.savetxt(f_w,norm_weights.reshape(len(context_weights),1),delimiter=',')
    f_w.write(','.join([str(i[0]) for i in norm_weights])+'\n')
    return context_avg




In [15]:
def filter_w(w,word2index,index2word,word_freq_f):
    #filter out words with freq less than 200, words with no letters in, and stopwords
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    index2word_filter={}
    word2index_filter={}
    index_filter2index=[]
    counter=0
    with open(word_freq_f) as f:
        for line in f:
            f_w=line.split()
            if len(f_w)>1:
                if f_w[1] in word2index and re.search('[a-zA-Z]',f_w[1])!=None and f_w[1] not in stopw :
                    #word2freq_nostop[f_w[1]]=f_w[0]
                    index_filter2index.append(word2index[f_w[1]])
                    word2index_filter[f_w[1]]=counter
                    index2word_filter[counter]=f_w[1]
                    counter+=1
    w_filter= w[index_filter2index,:]
    return w_filter,word2index_filter,index2word_filter

def rm_stopw_context(model):
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    
    model={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
    return model




In [24]:
def eval_chimera(chimeras_data_dir,num_sent,context_model,model_type,n_result,w,index2word,weight=False,w2entropy=None):
    with open(os.path.join(chimeras_data_dir,'weights_{0}_{1}_{2}'.format(num_sent,model_type,str(weight))),'w') as f_w:
        spearmans=[]
        
        num_indic='l'+str(num_sent)
        for root, subdir, fname in os.walk(chimeras_data_dir):
            for fn in fname:
                if fn.endswith('fixed.test.txt.punct')and 'l'+str(num_sent)==fn.split('.')[1]: #read in the test file
                    print (fn)
                    data=pd.read_csv(os.path.join(chimeras_data_dir,fn),delimiter='\t',header=None)
                    for index, row in data.iterrows():
                        golds=[]
                        model_predict=[]
                        #compute context representation
                        if weight!='learned':
                            context_avg=additive_model(f_w,row[1],'___', model_type,context_model,n_result,w,index2word,weight,w2entropy)


                        #cosine similarity with probe embedding
                        for gold,probe in zip(row[3].split(','),row[2].split(',')):
                            try:
                                cos=w[word2index[probe]].dot(context_avg)
                                if xp.isnan(cos):
                                    continue
                                else:
                                    model_predict.append(cos)
                                    golds.append(gold)
                                    
                            except KeyError as e:
                                print ("====warning key error for probe=====: {0}".format(e))
                        
                        sp=spearmanr(golds,model_predict)[0]
                        print ('spearman correlation is {0}'.format(sp))
                        if not math.isnan(sp):
                            spearmans.append(sp)
#         if len(golds)==len(model_predict):
#             print ('spearman correlation is {0}'.format(spearmanr(golds,model_predict)))
#         else:
#             print ('unequal length: gold {0}, model {1}'.format( len(golds),len(model_predict)))

        print ("AVERAGE RHO:",float(sum(spearmans))/float(len(spearmans)))

In [3]:
# a=np.array([1,2,3])
# b=a.reshape(len(a),1)
# c=(a+b)/2.0
# c=c/(sum(sum(c)))
# # print(sum(sum(c)))
# d=np.array([[1,1,1],[2,2,2],[3,3,3]])
# e=d.dot(d.T)
# print ('e',e)
# print ('c',c)
# e*c
# a
# context_embed= model.context2vec(['cats',',','dogs',',','snakes','are','animals'],2 )
# context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
# produce_top_n_simwords(context_embed=context_embed,index2word=index2word,w_filter=w,n_result=20)

In [18]:

if __name__=="__main__":
    
    #params read in
    if sys.argv[0]=='/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py':
#         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params'
#         model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.0'

#         model_type='context2vec'
#         context_rm_stopw=0
        model_param_file='../models/wiki_all.model/wiki_all.sent.split.model'
        model_type='skipgram'
        context_rm_stopw=1
        weight='lda'
        data='./eval_data/data-chimeras'

    else:
        if len(sys.argv) < 3:
            print >> sys.stderr, "Usage: %s <model_param_file> <model_type> <weight> <context_rm_stop>"  % (sys.argv[0])
            sys.exit(1)
        
        model_param_file = sys.argv[1]
        model_type=sys.argv[2]
        
        weight=int(sys.argv[3])
        context_rm_stop=int.argv[4]
        data =argv[5]
       
    
    
    #gpu setup 
    gpu = -1 # todo: make this work with gpu

    if gpu >= 0:
        cuda.check_cuda_available()
        cuda.get_device(gpu).use()    
    xp = cuda.cupy if gpu >= 0 else np
    
    # logging
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    
    #choose model type
    print ('read model....')
    if model_type=='context2vec':
        #read in model
        
        model_reader = ModelReader(model_param_file)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
    elif model_type=='skipgram':
        model = gensim.models.Word2Vec.load(model_param_file)
        w=model.wv.vectors
        #vector normalize for probe w embedding
        s = np.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model.wv.index2word
        word2index={key: model.wv.vocab[key].index for key in model.wv.vocab}
        
        
    #weight
    w2entropy=None
    if weight=='top_mutual_sim':
        #filter w : remove stop words and low freq words 
        print ('filter words for target....')
        print (len(w))
        w,word2index,index2word=filter_w(w,word2index,index2word,'word_freq_uwac')
        print (len(w))
        n_result = 20
    elif weight=='lda':
        print ('load vectors and entropy')
        w2entropy=pickle.load(open('../models/lda/w2entropy'))

    # remove context stop words
    if int(context_rm_stopw)==1:
        print ('filter words for context....')

        model=rm_stopw_context(model)
        
    

read model....
load vectors and entropy
filter words for context....


In [37]:
#read in data
if data.split('/')[-1]== 'data-chimeras':
        eval_chimera(data,6,model,model_type,20,w,index2word,weight,w2entropy)

dataset.l6.fixed.test.txt.punct
Canned sardines and  ___  between two slices of wholemeal bread and thinly spread Flora Original .
('sardines', 5.298317)
('two', 5.249666)
('slices', 5.298317)
('bread', 3.3919804)
('thinly', 5.298317)
('spread', 4.8279552)
weight mode lda not recognized
weight is 6
Erm ,  ___  , low fat dairy products , incidents of heart disease for those who have an olive oil rich diet .
('low', 5.1289616)
('fat', 2.5211575)
('dairy', 1.5041192)
('products', 4.917253)
('incidents', 3.456994)
('heart', 4.872587)
('disease', 4.9163074)
('olive', 2.0551832)
('oil', 5.135166)
('rich', 4.900546)
('diet', 3.3235273)
weight mode lda not recognized
weight is 11
These are the spermatangia , very  ___  ( length 150 micron ) crowded with spermatia .
('length', 4.852035)
('150', 3.4852371)
('crowded', 2.6370354)
weight mode lda not recognized
weight is 3
It ' s closest to mushroom but one of our friends guessed that there might be  ___  in it .
('closest', 3.975489)
('mushroom',

fritted: 0.7238623505944743
clinkers: 0.7155988776326325
drizzled: 0.7142341991724667
phyllo: 0.7137869805259337
griddles: 0.7136213765781902
precooked: 0.7135466687087597
puddled: 0.7128781521733227
lampblack: 0.7119990061712755
vacuumed: 0.7119775303776067
baghouse: 0.7119387059425535
gochujang: 0.7105590669316351
croutons: 0.7081176036791124
carburizing: 0.7076862749146023
trivalved: 0.7070425681419388
stovetop: 0.706997429782781
sludges: 0.7069950621746929
driers: 0.7059932711144822
pureed: 0.7058148942789265
cleaning: 0.7054811742443468
charcoals: 0.7054434617270212
spearman correlation is 0.811679449913
What never needs sharpening , is completely  ___  safe and is covered by a 25 year guarantee ?
('never', 5.185225)
('needs', 4.590453)
('sharpening', 5.298317)
('completely', 5.0272527)
('safe', 4.39557)
('covered', 4.763048)
('25', 4.934411)
('year', 5.1629496)
('guarantee', 2.690908)
weight mode lda not recognized
weight is 9
A large manor would also be quite self - sufficient w

hmmm: 0.6943780847772214
1977-2010: 0.6937285622456761
suchit: 0.6924632002982647
go: 0.6918420436177308
trivalved: 0.6891386816187244
crooned: 0.685314920088075
psyched: 0.6846605517897606
say: 0.6830957039292396
hollered: 0.6829716603222373
embraceable: 0.682508434948119
witchu: 0.6818747187499115
want: 0.6811744313333684
something: 0.6791047796020396
chucked: 0.6787512166359013
90ff90: 0.6773986024157225
uthup: 0.6768126676971549
luvin: 0.6766472164355528
fuckers: 0.6764418102521048
moaned: 0.676136514354864
feel: 0.6760827183753393
spearman correlation is 0.0289885517826
The new corps of McNeillstown scored their first success of the season by winning  ___  with the band gaining third overall placing .
('new', 5.2175474)
('corps', 3.7534285)
('scored', 2.5779448)
('first', 5.216248)
('success', 5.1666307)
('season', 4.8637543)
('winning', 4.6961093)
('band', 4.604214)
('gaining', 2.8773227)
('third', 5.1968365)
('overall', 4.8001003)
('placing', 3.902344)
weight mode lda not recogn

pterocliformesfamily: 0.6945030921894028
tichodromidae: 0.6943061606197762
flycatch: 0.6920817717137354
caracals: 0.6880891385443959
twolegs: 0.6869809807246767
anseriformesfamily: 0.6866812162828191
pteroclidae: 0.6866522504152045
cuculiformesfamily: 0.6857466048481906
solenodons: 0.6850806282504327
psittaciformesfamily: 0.6849120811619649
tegus: 0.6844727813757603
monals: 0.6839739105329802
peafowls: 0.6839512607240628
leafbirds: 0.6829610543304598
trogoniformesfamily: 0.6828938461134886
spurfowls: 0.6824312898751881
tyrannia: 0.682024589702052
apodiformesfamily: 0.6816901089524336
sittidae: 0.6814396998255288
caprimulgiformesfamily: 0.6810456350014131
spearman correlation is 0.714285714286
Upon arrival , enjoy a short trek to a small lake  ___  are frequently observed .
('arrival', 4.495202)
('enjoy', 3.7972026)
('short', 4.271385)
('trek', 5.298317)
('small', 5.220316)
('lake', 4.701524)
('frequently', 4.9173174)
('observed', 5.0211153)
weight mode lda not recognized
weight is 8
At

tichodromidae: 0.7103604312166546
flycatch: 0.7103367678267598
peafowls: 0.7034691561870049
pterocliformesfamily: 0.7014592325935464
subcanopy: 0.7005553835418848
spurfowls: 0.7003601676614608
monals: 0.6998151519137696
philodota: 0.699316220135573
otididae: 0.6965789902966392
sittidae: 0.6964830674396263
psittaciformesfamily: 0.6952852664596242
myrtillus: 0.6948605898539894
apodiformesfamily: 0.6942140764276741
geophytes: 0.6941669509745123
pademelons: 0.6939074283918515
cuculiformesfamily: 0.6938815765410964
streamsides: 0.6937347810480977
tragopans: 0.6937335468256586
leafbirds: 0.6936951943749234
pteroclidae: 0.6934764073247309
spearman correlation is 0.0289885517826
The RPF denies responsibility and says claims that it has  ___  are also untrue .
('denies', 5.298317)
('responsibility', 4.4047656)
('says', 4.9191)
('claims', 5.0510197)
('also', 5.266862)
('untrue', 5.298317)
weight mode lda not recognized
weight is 6
Once he ate the honey , he was a different  ___  and complimented

...

really: 0.6956313559123144
scraggly: 0.6939258468139411
wiggled: 0.6937338202887052
grownup: 0.6921172409206106
trivalved: 0.6912605628680317
squirming: 0.6902449400931875
pterocliformesfamily: 0.6898817866893621
longingly: 0.6892789897995149
decollated: 0.6870254742252793
90ff90: 0.6868912414571325
moaned: 0.686557550911248
forlornly: 0.6861442905582055
tichodromidae: 0.6861107182255461
alcoriza: 0.6860785614498147
standin: 0.6860491619655906
interneural: 0.6852806630830643
flycatch: 0.6852788442939578
thikkurissi: 0.6850141271463651
heartbreakingly: 0.685006306489017
moping: 0.6848369950624438
spearman correlation is 0.485714285714
It is one of the food plants for  ___  of Monarch and Queen butterflies .
('one', 5.2474523)
('food', 5.1340747)
('plants', 4.897327)
('butterflies', 5.298317)
weight mode lda not recognized
weight is 4
All the detainees were moved to the punishment block , which the detainees say is infested  ___  and other vermin .
('detainees', 5.298317)
('moved', 5.021

hummable: 0.7025285947837614
1977-2010: 0.7005727354362057
nithyasree: 0.6990416087511215
embraceable: 0.6983270990583945
jalota: 0.696474494096254
music: 0.6961611968020056
sevdalinka: 0.696014657107697
hischak: 0.6949051457014139
vocalese: 0.6938984119366587
serialist: 0.6932928416842813
coloristic: 0.6932285595352812
rabindrasangeet: 0.6930774576122196
vjing: 0.6924696466247804
amonkar: 0.6923506001635213
wondrously: 0.6921950141931901
sundararama: 0.6918922584090628
emote: 0.6918013563362527
hilye: 0.6916017335654571
nazms: 0.6915816679514841
maddalam: 0.6909090321376611
spearman correlation is 0.753702346348
A WIDOW whose arm was bitten off by a  ___  said yesterday she was sorry the creature was later killed .
('whose', 5.1855607)
('arm', 3.8737683)
('bitten', 5.298317)
('said', 5.084664)
('yesterday', 5.298317)
('sorry', 1.4387498)
('creature', 2.5497823)
('later', 5.240397)
('killed', 4.984482)
weight mode lda not recognized
weight is 9
It has been estimated that , of the thous

1977-2010: 0.7098118006213227
amonkar: 0.6918769246617509
going: 0.6914885434904317
nithyasree: 0.6913517046285249
so: 0.6908034016148201
really: 0.6903999844736709
embraceable: 0.6885615684110893
way: 0.688425500843371
psyched: 0.6879886874195515
kelucharan: 0.6873558876249426
bajnoksag: 0.6866496968764026
jalota: 0.68599422245797
gangubai: 0.6855533553004466
untz: 0.6844425310627381
indicud: 0.6842837925414057
trivalved: 0.6836985548949395
missin: 0.6833708543085664
ganjawala: 0.6826773550542318
witchu: 0.6825653552873617
thikkurissi: 0.6825021775249062
spearman correlation is 0.289885517826
I could hear a wailing  ___  over a faint beat , a melody filtered through hidden speakers .
('could', 5.1993427)
('hear', 3.9794571)
('faint', 2.4925992)
('beat', 4.1553597)
('melody', 2.6299345)
('filtered', 1.6349127)
('hidden', 4.0194583)
('speakers', 4.6231136)
weight mode lda not recognized
weight is 8
I imagine I was holding my  ___  case , kicking at the loose pebbles in the roadway .
('i

drizzled: 0.7255970543008685
phyllo: 0.7199174740676937
croutons: 0.7182048565535288
wontons: 0.714613349580139
pureed: 0.713631028592224
trivalved: 0.713594057146016
gochujang: 0.713202648633692
croquette: 0.7129915331979505
shortcrust: 0.7127847119398574
banchan: 0.71274701334454
cocoyam: 0.7126263348220938
sugared: 0.7109268474494737
quesadillas: 0.710853233797401
drinkware: 0.7108153158677605
vinaigrette: 0.7098771172993188
tteok: 0.7093510864982205
precooked: 0.7091444239136353
applesauce: 0.7087269917036452
sweetmeat: 0.7079976116962725
cutlets: 0.7070246259793578
spearman correlation is -0.405839724957
Be pleasant , don ' t carp , but act like a  ___  and draw out all the information you can .
('pleasant', 2.1782353)
('carp', 5.298317)
('act', 5.113785)
('like', 5.216198)
('draw', 4.3892345)
('information', 5.191952)
weight mode lda not recognized
weight is 6
Zest and juice the lemon ( I use a  ___  for the zest and a lemon squeezer for the juice ).
('juice', 2.0532694)
('lemon'

cocoyam: 0.7621231653412128
beans: 0.7584632703311143
cowpeas: 0.7533339430505896
potatoes: 0.7478718651306396
jawar: 0.7436354843518759
cashews: 0.7429149891213581
calamansi: 0.7423496116039121
sweetcorn: 0.7403205143208083
lychees: 0.7383442037348749
jowar: 0.7380340971786661
vegetables: 0.737343109664781
chayote: 0.7365000528321255
corn: 0.7364266654361165
cantaloupes: 0.7361682074576701
groundnuts: 0.7360342577554386
corncobs: 0.7355895791459983
grapefruits: 0.7349583506000399
manioc: 0.7348476568570687
wholemeal: 0.7342990824731325
gochujang: 0.7342538042178689
spearman correlation is -0.542857142857
Sharp  ___  skegs from the heads of bair could creep irritatingly up the inside of a sleeve , as I well remember .
('heads', 3.8339002)
('could', 5.1993427)
('creep', 5.298317)
('inside', 4.985652)
('sleeve', 1.2694514)
('well', 5.110713)
('remember', 2.6984286)
weight mode lda not recognized
weight is 7
And one day when she was working in her little garden slithering between the  ___